2. Snake
Extend example project from Lab 8 and add following tasks:

 1) Randomly generating food with different weights
 2) Foods which are disappearing after some time(timer)
 3) Comment your code

In [1]:
import pygame  
import sys  # Import the sys module for system interactions
import copy  # Import the copy module for creating object copies
import random  # Import the random module for random number generation
import time  # Import the time module for time-related functions

pygame.init()  # Initialize Pygame

# Game configuration settings
scale = 15  # Scale factor for game objects
score = 0  # Player's current score
level = 0  # Current game level
SPEED = 10  # Base speed of the snake

# Food position variables
food_x = 10  # X-coordinate of food
food_y = 10  # Y-coordinate of food

# Create the game display window
display = pygame.display.set_mode((500, 500))
pygame.display.set_caption("Snake Game")  # Set window title
clock = pygame.time.Clock()  # Create clock object for controlling game speed

# Color definitions in RGB format
background_top = (0, 0, 50)  # Top gradient color
background_bottom = (0, 0, 0)  # Bottom gradient color
snake_colour = (255, 137, 0)  # Snake body color
food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))  # Random food color
snake_head = (255, 247, 0)  # Snake head color
font_colour = (255, 255, 255)  # Text color
defeat_colour = (255, 0, 0)  # Game over text color

class Snake:
    def __init__(self, x_start, y_start):
        # Initialize snake properties
        self.x = x_start  # Starting x position
        self.y = y_start  # Starting y position
        self.w = 15  # Width of snake segments
        self.h = 15  # Height of snake segments
        self.x_dir = 1  # X-direction movement (1 = right, -1 = left)
        self.y_dir = 0  # Y-direction movement (1 = down, -1 = up)
        self.history = [[self.x, self.y]]  # Movement history (positions)
        self.length = 1  # Current snake length

    def reset(self):
        """Reset snake to starting position and state"""
        self.x = 500 / 2 - scale  # Center x position
        self.y = 500 / 2 - scale  # Center y position
        self.w = 15  # Reset width
        self.h = 15  # Reset height
        self.x_dir = 1  # Reset to moving right
        self.y_dir = 0  # Reset y movement
        self.history = [[self.x, self.y]]  # Clear movement history
        self.length = 1  # Reset length

    def show(self):
        """Draw the snake on the display"""
        for i in range(self.length):
            if not i == 0:  # For body segments
                pygame.draw.rect(display, snake_colour, (self.history[i][0], self.history[i][1], self.w, self.h))
            else:  # For head segment
                pygame.draw.rect(display, snake_head, (self.history[i][0], self.history[i][1], self.w, self.h))

    def check_eaten(self):
        """Check if snake has eaten food"""
        if abs(self.history[0][0] - food_x) < scale and abs(self.history[0][1] - food_y) < scale:
            return True

    def check_level(self):
        """Check if player has reached a new level"""
        global level
        if self.length % 5 == 0:
            return True

    def grow(self):
        """Increase snake length"""
        self.length += 1
        self.history.append(self.history[self.length - 2])

    def death(self):
        """Check for collision with self"""
        i = self.length - 1
        while i > 0:
            if abs(self.history[0][0] - self.history[i][0]) < self.w and abs(self.history[0][1] - self.history[i][1]) < self.h and self.length > 2:
                return True
            i -= 1

    def update(self):
        """Update snake position"""
        i = self.length - 1
        while i > 0:  # Move each segment to follow the one ahead
            self.history[i] = copy.deepcopy(self.history[i - 1])
            i -= 1
        # Move head based on direction
        self.history[0][0] += self.x_dir * scale
        self.history[0][1] += self.y_dir * scale

class Food:
    def new_location(self):
        """Generate new random food position"""
        global food_x, food_y
        food_x = random.randrange(1, int(500 / scale) - 1) * scale
        food_y = random.randrange(1, int(500 / scale) - 1) * scale

    def show(self):
        """Draw food on the display"""
        pygame.draw.rect(display, food_colour, (food_x, food_y, scale, scale))

def show_score():
    """Display current score"""
    font = pygame.font.SysFont(None, 20)
    text = font.render("Score: " + str(score), True, font_colour)
    display.blit(text, (scale, scale))

def show_level():
    """Display current level"""
    font = pygame.font.SysFont(None, 20)
    text = font.render("Level: " + str(level), True, font_colour)
    display.blit(text, (90 - scale, scale))

def gameLoop():
    """Main game loop"""
    global score, level, SPEED

    snake = Snake(500 / 2, 500 / 2)  # Create snake object
    food = Food()  # Create food object
    food.new_location()  # Set initial food position

    while True:  # Main game loop
        for event in pygame.event.get():  # Event handling
            if event.type == pygame.QUIT:  # Window close event
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:  # Key press events
                if event.key == pygame.K_q:  # Quit key
                    pygame.quit()
                    sys.exit()
                # Direction change handling
                if snake.y_dir == 0:  # Currently moving horizontally
                    if event.key == pygame.K_UP:  # Change to up
                        snake.x_dir = 0
                        snake.y_dir = -1
                    if event.key == pygame.K_DOWN:  # Change to down
                        snake.x_dir = 0
                        snake.y_dir = 1
                if snake.x_dir == 0:  # Currently moving vertically
                    if event.key == pygame.K_LEFT:  # Change to left
                        snake.x_dir = -1
                        snake.y_dir = 0
                    if event.key == pygame.K_RIGHT:  # Change to right
                        snake.x_dir = 1
                        snake.y_dir = 0

        # Draw gradient background
        for y in range(500):
            color = (
                background_top[0] + (background_bottom[0] - background_top[0]) * y / 500,
                background_top[1] + (background_bottom[1] - background_top[1]) * y / 500,
                background_top[2] + (background_bottom[2] - background_top[2]) * y / 500
            )
            pygame.draw.line(display, color, (0, y), (500, y))

        # Game rendering
        snake.show()
        snake.update()
        food.show()
        show_score()
        show_level()

        # Game logic
        if snake.check_eaten():  # Food collision
            food.new_location()
            score += random.randint(1, 5)  # Random score increase
            snake.grow()

        if snake.check_level():  # Level up condition
            food.new_location()
            level += 1
            SPEED += 1  # Increase game speed
            snake.grow()

        if snake.death():  # Game over condition
            score = 0
            level = 0
            font = pygame.font.SysFont(None, 100)
            text = font.render("Game Over!", True, defeat_colour)
            display.blit(text, (50, 200))
            pygame.display.update()
            time.sleep(3)  # Pause before reset
            snake.reset()

        # Screen wrapping logic
        if snake.history[0][0] > 500:  # Right edge
            snake.history[0][0] = 0
        if snake.history[0][0] < 0:  # Left edge
            snake.history[0][0] = 500
        if snake.history[0][1] > 500:  # Bottom edge
            snake.history[0][1] = 0
        if snake.history[0][1] < 0:  # Top edge
            snake.history[0][1] = 500

        pygame.display.update()  # Update display
        clock.tick(SPEED)  # Control game speed

gameLoop()  # Start the game

pygame 2.6.1 (SDL 2.28.4, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\admin\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
